In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 0#np.array([0, 0, 0, 1])
    elif i % 5  == 0: return 1 #np.array([0, 0, 1, 0])
    elif i % 3  == 0: return 2#np.array([0, 1, 0, 0])
    else:             return 3#np.array([1, 0, 0, 0])

def invert(label, sample):
    if label == 3:
        return sample
    else:
        return {0:'fizzbuzz', 1:'fizz',2:'buzz'}[label]

In [3]:
def generate_training(num_digits, therange):
    for i in range(*therange):
        yield binary_encode(i, num_digits)
names=[str(i) for i in range(10)]
train_x = pd.DataFrame((i for i in generate_training(10,(101,1024))), columns=names)
test_range = (0,100)
train_range = (101,1024)
train_y = np.asarray([fizz_buzz_encode(i) for i in range(*train_range)])
test_x = pd.DataFrame((i for i in generate_training(10,test_range)),columns=names)
test_y = np.asarray([fizz_buzz_encode(i) for i in range(*test_range)])

def train_input_fn(features, labels, batch_size=20):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))#.map(namer)
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat(100).batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    data, labels = iterator.get_next()
    return data, labels
def test_input_fn(features, labels, batch_size=20):
    """An input function for evaluation or prediction"""
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))#.map(namer)
    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    # Return the dataset.
    iterator = dataset.make_one_shot_iterator()
    data, labels = iterator.get_next()
    return data, labels

In [4]:
def main(train_x, train_y, test_x, test_y):
    my_feature_columns = [tf.feature_column.numeric_column(key=str(k))
                          for k in range(10)]

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # The model must choose between 4 classes
        n_classes=4,
        # Two hidden layers of 100 nodes each.
        hidden_units=[100, 100]
        )

    # Train the Model.
    classifier.train(input_fn=lambda:train_input_fn(train_x, train_y))

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:test_input_fn(test_x, test_y))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
    predictions = classifier.predict(
        input_fn=lambda:test_input_fn(test_x,test_y))
    formatted = [invert(pred.get('class_ids')[0], idx)
                for idx, pred in enumerate(predictions)]
    expected = [invert(fizz_buzz_encode(i),i) for i, p in enumerate(test_y)]
    return expected, formatted
e, f = main(train_x, train_y, test_x, test_y)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_num_worker_replicas': 1, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_is_chief': True, '_master': '', '_save_checkpoints_secs': 600, '_service': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f91690f28>, '_model_dir': '/tmp/tmpjbvl8flc', '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpjbvl8flc/model.ckpt.
INFO:tensorflow:loss = 25.68321, step = 1
INFO:tensorflow:global_step/sec: 351.629
INFO:tensorflow:loss = 24.695992, step = 101 (0.287 sec)
INFO:tensorflow:global_step/sec: 416.926
INFO:tensorflow:loss = 18.26225, step = 201 (0.239 sec)
INFO:tensorflow:global_step/sec: 512.33
INFO:tensorflow:loss = 22

In [6]:
for expected, got in zip(e,f):
    print('answer is {}, neural net predicted {} {}'.format(expected, got, ('wrong' if expected!=got else '')))

answer is fizzbuzz, neural net predicted buzz wrong
answer is 1, neural net predicted 1 
answer is 2, neural net predicted 2 
answer is buzz, neural net predicted buzz 
answer is 4, neural net predicted 4 
answer is fizz, neural net predicted 5 wrong
answer is buzz, neural net predicted buzz 
answer is 7, neural net predicted 7 
answer is 8, neural net predicted 8 
answer is buzz, neural net predicted buzz 
answer is fizz, neural net predicted fizz 
answer is 11, neural net predicted 11 
answer is buzz, neural net predicted buzz 
answer is 13, neural net predicted 13 
answer is 14, neural net predicted 14 
answer is fizzbuzz, neural net predicted fizzbuzz 
answer is 16, neural net predicted 16 
answer is 17, neural net predicted 17 
answer is buzz, neural net predicted buzz 
answer is 19, neural net predicted 19 
answer is fizz, neural net predicted fizz 
answer is buzz, neural net predicted buzz 
answer is 22, neural net predicted 22 
answer is 23, neural net predicted 23 
answer is b